In [16]:
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
import math
import random as r
import scipy.stats
import os.path

In [17]:
import sys
sys.path.append('/home/machado.35/projects/notebooks')
from deprojectGalaxy import deproject

def deprojectMap(image, errImage, galRA, galDEC, pa, incl, galDist):
    # convert pixel map to x' & y' and ra & dec
    # image and errImage are the image and noise file pathways
    # galRA, galDEC, pa, incl, galDist are all found within the PHANGS datatabels

    hdulist   = pyfits.open(image)
    intMap    = hdulist[0].data
    hdulist2  = pyfits.open(errImage)
    errMap    = hdulist2[0].data
        
    wcs      = WCS(hdulist[0].header, naxis=2)
    naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid     = np.indices((naxis[1],naxis[0]))
    ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)

    centerCoord = [galRA, galDEC]
    
    #deproject ra and dec to dx and dy
    radius, projang, dx, dy = deproject(center_coord=centerCoord, incl=incl, pa=pa, ra=ra, dec=dec, return_offset=True)
    
    # show galaxy image
    # plt.imshow(map, origin='lower', interpolation='nearest', zorder=1)

    #flatten data structures 
    f_int  = intMap.flatten()
    f_err  = errMap.flatten()
    f_ra   = ra.flatten()
    f_dec  = dec.flatten()    
    f_dx   = dx.flatten()
    f_dy   = dy.flatten()       
    
    #remove nans
    keep  = np.where(np.isfinite(f_int))
    inten = f_int[keep]
    err   = f_err[keep]
    ra    = f_ra[keep]
    dec   = f_dec[keep]
    dx    = f_dx[keep]
    dy    = f_dy[keep]
    
    # calculate SNR
    SNR = []
    for i in range(len(inten)):
        if err[i] == 0.0:
            SNR.append(0.0)
        elif inten[i] < 0.0:
            SNR.append(0.0)           
        else:
            SNR.append(inten[i]/err[i])

    return(inten, err, SNR, ra, dec, dx, dy)

In [18]:
# This block of code contains the supporting functions to generate 
# a random distribution, you should not need to call any of these 
# funcitons specifically.

def arraySort(variable, distance):
    # sorts variable and distance list by shortest distance
    # variable, distance: lists holding measurement of variable and distance to that measurement
    pattern = distance.argsort()
    dist = distance[pattern]
    var = variable[pattern]
    return (var, dist)

def findNearest(varArray, value, distArray):
    # sorts variable by distance, returning the closest that has the given value 
    # varArray, distArray: arrays holding measurement of variable and distance to that measurement
    # value: float. 
    var, dist = arraySort(varArray, distArray)
    ind = np.where(var >= value)
    
    if len(dist[ind]) > 0:
        nearestVal  = np.argmin(dist[ind])
        nearestDist = dist[ind][nearestVal] 
        varVal     = var[ind][nearestVal]

    else:
        varVal     = float('nan')
        nearestDist = float('nan')
        
    return(varVal, nearestDist)
    
def printNearest(inten, SNR, dist_kpc, value, SNRcutoff = 0.0):
    # returns distance to the nearest molecular cloud and the intensity value found
    
    #apply SNR cutoff
    intenCut, dist_kpcCut = [],[]
    for i in range(len(inten)):
        if (SNR[i] >= SNRcutoff):
            intenCut.append(inten[i])
            dist_kpcCut.append(dist_kpc[i])
    
    valFound, nearestMC = findNearest(np.array(intenCut), value, np.array(dist_kpcCut))
    
    return(nearestMC, valFound)      

def distanceCalculator(x1, x2, y1, y2, galDist):
    #calculate distance between two points (in kpc)
    #x1, y1 = xprime and yprime, x2, y2 = SN coords, dist = distance to galaxy (kpc)
    d = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    x = galDist * np.tan(d*np.pi/180.0)
    return(x)

def normalize(weightsArray):
    # takes a list of weights and normalizes them
    prob_factor = 1 / sum(weightsArray)
    return [prob_factor * p for p in weightsArray]


In [19]:
# This block of code contains the random function, 
# which performs your random placement experiment using the 
# support functions above.

def random(numTests, weightsArray, dx, dy, ra, dec, galDist):

    # takes number of tests, array of weights, deprojected and 
    #       official coordinates, and distance to galaxy...and returns
    #       list of numTests # of random values, their nearest distance, 
    #       and their randomly generated coordinates
    # numTests: int , number of randomly generated values
    # weightsArray: list of weights 
    # dx&dy:  deprojected coordinates
    # ra&dec: official coordinates 
    # galDist: distance to galaxy in kpc
    val, dist, randX, randY, randRA, randDEC = [],[],[],[],[],[]
    
    for i in range(numTests):
        
        total = sum(weightsArray)
        prob  = weightsArray/total 
        prob  = normalize(prob)
        
        nX = len(dx)  
        indicies = np.arange(nX, dtype=int)
        randInt = np.random.choice(indicies)

        rX   = dx[randInt]
        rY   = dy[randInt]
        rRA  = ra[randInt]
        rDEC = dec[randInt]

        distRand = []
        distRand = distanceCalculator(dx, rX, dy, rY, galDist)

        # set sorting cutoff
        intenCutoff = 0
        
        distance, value = printNearest(inten, SNR, distRand, intenCutoff, SNRcutoff=0)
        val.append(value)
        dist.append(distance)
        randX.append(rX)
        randY.append(rY)
        randRA.append(rRA)
        randDEC.append(rDEC)
    return(val, dist, randX, randY, randRA, randDEC)

In [ ]:
import sys
sys.path.append('/home/machado.35/projects/notebooks')
from deprojectGalaxy import deproject


def gen_random(image, errImage, galRA, galDEC, pa, incl, galDist, numTests)
def deprojectMap(image, errImage, galRA, galDEC, pa, incl, galDist):
    # convert pixel map to x' & y' and ra & dec
    # image and errImage are the image and noise file pathways
    # galRA, galDEC, pa, incl, galDist are all found within the PHANGS datatabels

    hdulist   = pyfits.open(image)
    intMap    = hdulist[0].data
    hdulist2  = pyfits.open(errImage)
    errMap    = hdulist2[0].data
        
    wcs      = WCS(hdulist[0].header, naxis=2)
    naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid     = np.indices((naxis[1],naxis[0]))
    ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)

    centerCoord = [galRA, galDEC]
    
    #deproject ra and dec to dx and dy
    radius, projang, dx, dy = deproject(center_coord=centerCoord, incl=incl, pa=pa, ra=ra, dec=dec, return_offset=True)
    
    # show galaxy image
    # plt.imshow(map, origin='lower', interpolation='nearest', zorder=1)

    #flatten data structures 
    f_int  = intMap.flatten()
    f_err  = errMap.flatten()
    f_ra   = ra.flatten()
    f_dec  = dec.flatten()    
    f_dx   = dx.flatten()
    f_dy   = dy.flatten()       
    
    #remove nans
    keep  = np.where(np.isfinite(f_int))
    inten = f_int[keep]
    err   = f_err[keep]
    ra    = f_ra[keep]
    dec   = f_dec[keep]
    dx    = f_dx[keep]
    dy    = f_dy[keep]
    
    # calculate SNR
    SNR = []
    for i in range(len(inten)):
        if err[i] == 0.0:
            SNR.append(0.0)
        elif inten[i] < 0.0:
            SNR.append(0.0)           
        else:
            SNR.append(inten[i]/err[i])

    return(inten, err, SNR, ra, dec, dx, dy)





# This block of code contains the random function, 
# which performs your random placement experiment using the 
# support functions above.

def random(numTests, weightsArray, dx, dy, ra, dec, galDist):

    # takes number of tests, array of weights, deprojected and 
    #       official coordinates, and distance to galaxy...and returns
    #       list of numTests # of random values, their nearest distance, 
    #       and their randomly generated coordinates
    # numTests: int , number of randomly generated values
    # weightsArray: list of weights 
    # dx&dy:  deprojected coordinates
    # ra&dec: official coordinates 
    # galDist: distance to galaxy in kpc
    val, dist, randX, randY, randRA, randDEC = [],[],[],[],[],[]
    
    for i in range(numTests):
        
        total = sum(weightsArray)
        prob  = weightsArray/total 
        prob  = normalize(prob)
        
        nX = len(dx)  
        indicies = np.arange(nX, dtype=int)
        randInt = np.random.choice(indicies)

        rX   = dx[randInt]
        rY   = dy[randInt]
        rRA  = ra[randInt]
        rDEC = dec[randInt]

        distRand = []
        distRand = distanceCalculator(dx, rX, dy, rY, galDist)

        # set sorting cutoff
        intenCutoff = 0
        
        distance, value = printNearest(inten, SNR, distRand, intenCutoff, SNRcutoff=0)
        val.append(value)
        dist.append(distance)
        randX.append(rX)
        randY.append(rY)
        randRA.append(rRA)
        randDEC.append(rDEC)
    return(val, dist, randX, randY, randRA, randDEC)

In [20]:
def distcalc(coords, source, res):
    fp = '/home/machado.35/projects/intro/sources/'+source+'/'+source+'_12m+7m+tp_co21_'+str(res)+'pc_props.fits.bz2'
    if os.path.isfile(fp):
        tab = Table.read(fp)
        RA = coords[0]
        DEC = coords[1]
        mean_dist = np.zeros([4,3])
        percentiles = np.zeros([4,15])
        perc = [5,16,50,84,95]
        dist = np.zeros((len(RA),len(RA)))
        distance = np.array(tab['DISTANCE_PC'])
        mindist = np.zeros(len(RA)) #stores distance to nearest neighbor
        mindist2 = np.zeros(len(RA)) #stores distance to 2nd nearest neighbor
        mindist3 = np.zeros(len(RA)) #stores distance to 3rd nearest neighbor
        nn = np.zeros(len(RA)) #stores CloudNum of nearest neighbor
        nn2 = np.zeros(len(RA)) #stores CloudNum of 2nd nearest neighbor
        nn3 = np.zeros(len(RA)) #stores CloudNum of 3rd nearest neighbor
        k = 0
        j = 0
        while k < len(RA):
            for j in range(len(RA)):
                dist[k,j] = np.sqrt(np.square(RA[k] - RA[j]) + np.square(DEC[k] - DEC[j]))
            k+=1
        dist[dist == 0] = np.nan
        k=0
        for k in range(len(RA)):
            ind = np.where(dist[k] == np.nanmin(dist[k])) #index of nearest neighbor
            ind2 = np.where(dist[k] == np.partition(dist[k], 1)[1]) #index of 2nd nearest neighbor
            ind3 = np.where(dist[k] == np.partition(dist[k], 2)[2]) #index of 3rd nearest neighbor
            mindist[k] = np.deg2rad(dist[k,int(ind[0][0])])*distance[0]
            mindist2[k] = np.deg2rad(dist[k,int(ind2[0][0])])*distance[0]
            mindist3[k] = np.deg2rad(dist[k,int(ind3[0][0])])*distance[0]
    return(mindist, mindist2, mindist3)

In [21]:
####PREP BLOCK FOR LOOP#####
sources = pd.read_csv('/home/machado.35/projects/intro/scripts/sources.csv')
sources = list(sources['sources'])

####FIX AFTER 1K RUN
res = [60,90,120,150]

tab_loc = '/home/machado.35/projects/intro/phangs_sample_table_v1p6.fits'
t = Table.read(tab_loc)

mask_fp = '/home/machado.35/projects/masks/coverage/'


In [23]:
##### NUMBER OF DESIRED RANDOM PLACEMENTS #####
n = 1e4

In [ ]:
for k in range(len(res)):
    for i in range(len(sources)):
        if os.path.isfile('/data/rubin/machado.35/phangs/rand_results/'+sources[i]+'_1e'+str(e_num)+'_2dcov'+'_'+str(res[k])+'pc.csv') == False:
            image = mask_fp+sources[i]+'_12m+7m+tp_co21_'+str(res[k])+'pc_coverage2d.fits'
            errImage = '/home/machado.35/projects/masks/exp/'+sources[i]+'_'+str(res[k])+'pc_exp_mask.fits'
            if os.path.isfile(image):
                row = int(np.where(t['name'] == sources[i])[0])
                cat_fp = '/home/machado.35/projects/intro/sources/'+sources[i]+'/'+sources[i]+'_'+str(res[k])+'pc_cloud_stats.csv'
                if os.path.isfile(cat_fp):
                    obj = deprojectMap(image=image, errImage=errImage, galRA=t[row]['orient_ra'], galDEC=t[row]['orient_dec'],
                                      pa=t[row]['orient_posang'], incl=t[row]['orient_incl'], galDist=t[row]['dist'])
                    cat = pd.read_csv(cat_fp)
                    numTests = len(cat['min_dist']) #Number of placements per loop (equal to num of real clouds)
                    numloop = math.ceil(n/numTests)
                    inten=obj[0]
                    SNR = obj[2]
                    dx = obj[5]
                    dy = obj[6]
                    galDist = t[row]['dist']
                    coords = [[],[]]
                    rand_first, rand_second, rand_third = [], [], []
                    df = pd.DataFrame()
                    source = sources[i]
                    for j in range(numloop):
                        tests = random(numTests, inten, dx, dy, obj[3], obj[4], galDist)
                        coords[0] = tests[4]
                        coords[1] = tests[5]
                        z = distcalc(coords, source, res[k])
                        rand_first.extend(z[0])
                        rand_second.extend(z[1])
                        rand_third.extend(z[2])
                        #Save / Store each individual simulation run
                        e_num = len(str(int(n)))-1 ###for file naming 1e5/1e6,etc
                        df['run_'+str(j)+'_first'] = z[0]
                        df['run_'+str(j)+'_second'] = z[1]
                        df['run_'+str(j)+'_third'] = z[2]
                    df.to_csv('/data/rubin/machado.35/phangs/rand_results/'+sources[i]+'_1e'+str(e_num)+'_2dcov'+'_'+str(res[k])+'pc.csv')